In [8]:
setwd("/liulab/galib/mm_dlbcl_manuscript/")
library(tidyverse)
library(Seurat)
library(harmony)
library(dplyr)
library(DoubletFinder)
library(rBCS)
library(ComplexHeatmap)
source("./scripts/scplot.R")

Warning message:
“package ‘ComplexHeatmap’ was built under R version 4.1.3”
Loading required package: grid

ComplexHeatmap version 2.10.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))




In [ ]:
merged_object<- readRDS("data/objects/merged/merged_res1.5_obj.rds")
cd3_pos_cd4_neg<- readRDS("data/objects/cd3_pos_cd4_neg/cd3_pos_cd4_neg_res1.5_obj.rds")
cd3_pos_cd8_neg<- readRDS("data/objects/cd3_pos_cd8_neg/cd3_pos_cd8_neg_res1.5_obj.rds")
B_cell<- readRDS("data/objects/B_cell/B_cell_res1.5_obj.rds")

## Remove clusters with less than 41 cells

In [ ]:
cd3_pos_cd4_neg_clusters_41_exclusive<- 
cluster_remain_41_exclusive(cd3_pos_cd4_neg, "cd3_pos_cd4_neg")

cd3_pos_cd8_neg_clusters_41_exclusive<- 
cluster_remain_41_exclusive(cd3_pos_cd8_neg, "cd3_pos_cd8_neg")

B_cell_clusters_41_exclusive<- 
cluster_remain_41_exclusive(B_cell, "B_cell")


remove_clusters(cd3_pos_cd4_neg, cd3_pos_cd4_neg_clusters_41_exclusive, 
                space = "cd3_pos_cd4_neg", space_name = "Cd3+ Cd4-")
  
remove_clusters(cd3_pos_cd8_neg, cd3_pos_cd8_neg_clusters_41_exclusive, 
                space = "cd3_pos_cd8_neg", 
                space_name = "Cd3+ Cd8-")

remove_clusters(B_cell, B_cell_clusters_41_exclusive, 
                space = "B_cell", 
                space_name = "B cell")

## Subclustering specific clusters in each space

In [2]:
cd3_pos_cd4_neg<- readRDS("./data/objects/cd3_pos_cd4_neg_clusters_removed.rds")
cd3_pos_cd8_neg<- readRDS("data/objects/cd3_pos_cd8_neg_clusters_removed.rds")
B_cell<- readRDS("data/objects/B_cell_clusters_removed.rds")

### Subclustering c4 in cd3_pos_cd8_neg

In [ ]:
cd3_pos_cd8_neg_4<- subset(cd3_pos_cd8_neg, seurat_clusters == "4")

cd3_pos_cd8_neg_4<- cd3_pos_cd8_neg_4 %>%
NormalizeData(normalization.method = "LogNormalize", scale.factor = 10000)  %>% 
FindVariableFeatures(selection.method = "vst", nfeatures = 2000)  %>% 
ScaleData(vars.to.regress = "percent.mt") %>% 
RunPCA(npc = 50) %>%
RunHarmony(group.by.vars = "pool_id")

ElbowPlot(cd3_pos_cd8_neg_4, ndims = 50)

cd3_pos_cd8_neg_4 <- cd3_pos_cd8_neg_4  %>% 
FindNeighbors(reduction = "harmony", dims = 1:50) %>%
    FindClusters(resolution = 0.05)

DimPlot(cd3_pos_cd8_neg_4, reduction = "umap", 
        group.by = "seurat_clusters",
        pt.size = .1, label = TRUE, 
        repel = TRUE, order=TRUE, combine = TRUE)

ggsave("./results/figures/2_cd3_pos_cd8_neg_cluster4_subcluster_umap.pdf", width = 6, height = 3.5)


c4_markers<- presto::wilcoxauc(cd3_pos_cd8_neg_4, 'seurat_clusters', assay = 'data')
c4_top_20_markers<- top_markers(c4_markers, n = 20, auc_min = 0.55, pct_in_min = 20, pct_out_max = 20)
c4_top_markers_unique<- c4_top_20_markers %>%
  select(-rank) %>% 
  unclass() %>% 
  stack() %>%
  pull(values) %>%
  unique() %>%
   .[!is.na(.)]

c4_top_markers_unique  %>% length()

c4_mat<- cd3_pos_cd8_neg_4[["RNA"]]@data[c4_top_markers_unique, ] %>% as.matrix()
## scale the rows
c4_mat<- t(scale(t(c4_mat)))
c4_mat_cluster<- cd3_pos_cd8_neg_4@meta.data$seurat_clusters

quantile(c4_mat, c(0.1, 0.5, 0.95))
col_fun = circlize::colorRamp2(c(-1, 0, 2), viridis(3))

heat_c4<- Heatmap(c4_mat, name = "Expression",  
        column_split = factor(c4_mat_cluster),
        cluster_columns = TRUE,
        show_column_dend = FALSE,
        cluster_column_slices = TRUE,
        column_title_gp = gpar(fontsize = 12),
        column_gap = unit(0.5, "mm"),
        cluster_rows = TRUE,
        show_row_dend = FALSE,
        col = col_fun,
        row_names_gp = gpar(fontsize = 12),
        column_title_rot = 90,
        top_annotation = HeatmapAnnotation(foo = anno_block(gp = gpar(fill = scales::hue_pal()(22)),
                                           labels_gp = gpar(col = "white", fontsize = 10))),
        show_column_names = FALSE,
        use_raster = TRUE,
        raster_quality = 4)

heat_c4

pdf(here("results/figures/2_cd3_pos_cd8_neg_cluster4_marker_gene_heatmap.pdf"), width = 6, height = 8)
draw(heat_c4) 
dev.off()

In [ ]:
cluster_4_idents<- Idents(cd3_pos_cd8_neg_4)
cluster_4_a<- names(cluster_4_idents)[cluster_4_idents == 0]
cluster_4_b<- names(cluster_4_idents)[cluster_4_idents == 1]

cluster_df<- cbind(cd3_pos_cd8_neg@meta.data, barcode = colnames(cd3_pos_cd8_neg)) %>%
  mutate(seurat_clusters = as.character(seurat_clusters)) %>%
  mutate(seurat_clusters_sub = case_when(
    barcode %in% cluster_4_a ~ "4_a",
    barcode %in% cluster_4_b ~ "4_b",   
    TRUE ~ seurat_clusters
  ))

cluster_df$seurat_clusters_sub  %>% table()
cluster_df$seurat_clusters  %>% table()

In [ ]:
cd3_pos_cd8_neg_cluster_levels<- cluster_df$seurat_clusters_sub  %>% table() %>% names() %>% str_sort(numeric = TRUE)

cd3_pos_cd8_neg$seurat_clusters_old<- cd3_pos_cd8_neg$seurat_clusters


cd3_pos_cd8_neg$seurat_clusters <- factor(x = cluster_df$seurat_clusters_sub, 
                                              levels = cd3_pos_cd8_neg_cluster_levels)

cd3_pos_cd8_neg$seurat_clusters_old  %>% table()
cd3_pos_cd8_neg$seurat_clusters  %>% table

In [ ]:
DimPlot(cd3_pos_cd8_neg, reduction = "umap", group.by = "seurat_clusters",
        pt.size = .1, label = TRUE, repel = TRUE, order=TRUE, combine = TRUE)

ggsave("./results/figures/2_cd3_pos_cd8_neg_subcluster_umap.pdf", width = 12, height = 8)

saveRDS(cd3_pos_cd8_neg, "./data/objects/cd3_pos_cd8_neg_c4_subclustered.rds")
ExportSeurat(cd3_pos_cd8_neg, "./data/objects/cd3_pos_cd8_neg_c4_subclustered.bcs", overwrite=TRUE)

### remove c22 and c25 from cd3_pos_cd8_neg

In [ ]:
cd3_pos_cd8_neg_clusters<- cd3_pos_cd8_neg$seurat_clusters  %>% table()  %>% names()
cd3_pos_cd8_neg_clusters
cd3_pos_cd8_neg_new<- cd3_pos_cd8_neg_clusters[-c(length(cd3_pos_cd8_neg_clusters), length(cd3_pos_cd8_neg_clusters)-2)]
cd3_pos_cd8_neg_new

remove_clusters(cd3_pos_cd8_neg, obj_clusters = cd3_pos_cd8_neg_new, 
                space = "cd3_pos_cd8_neg", fsuffix = "c22_c25_removed")

### remove c26 c28 c35 from B-cell

In [ ]:
all_clusters<- B_cell$seurat_clusters  %>% table()  %>% names()
remain_clusters<- all_clusters[!all_clusters %in% c(26,28,35)]
remove_clusters(obj = B_cell, obj_clusters = remain_clusters, space = "B_cell", fsuffix = "c26_c28_c35_removed")